In [ ]:
import pandas as pd
import numpy as np
#read in quantified data
growth = pd.read_csv('yeast_growth.tsv', delimiter='\t')

In [ ]:
#dimension reduction
!pip install --upgrade scikit-learn
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
tsne = TSNE(n_components = 2, random_state = 0, init='random')
fit = tsne.fit_transform(growth.iloc[:, 3:])

In [ ]:
tsne_results = pd.DataFrame(fit, columns=['tsne1', 'tsne2'])

In [ ]:
#visualize t-sne result
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
color = le.fit_transform(growth['Nutrient'])
new_color = []
for c in color:
    if c==0:
        new_color.append('red')
    elif c==1:
        new_color.append('green')
    elif c==2:
        new_color.append('blue')
    elif c==3:
        new_color.append('purple')
    elif c==4:
        new_color.append('yellow')
    else:
        new_color.append('black')
new_color2 = ["green","red","purple","yellow","blue","black"]
plt.scatter(tsne_results['tsne1'], tsne_results['tsne2'], c=new_color,s = growth['Rate']*500)
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=condition, 
                               markerfacecolor=color, markersize=10) 
                   for condition, color in zip(growth['Nutrient'].unique(), 
                                               new_color2)]

plt.legend(handles=legend_elements, loc="upper right")
plt.show()

In [ ]:
#pca
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(growth.iloc[:, 3:])
pca_results = pca.transform(growth.iloc[:, 3:])
plt.scatter(pca_results[:, 0], pca_results[:, 1],c=new_color,s = growth['Rate']*500)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Transformed data')

legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=condition, 
                               markerfacecolor=color, markersize=10) 
                   for condition, color in zip(growth['Nutrient'].unique(), 
                                               new_color2)]

plt.legend(handles=legend_elements, loc="upper right")
plt.show()

In [ ]:
#DE analysis
# your code here (yielding count of significantly DE'ed genes)
library(pbapply)
library(dplyr)
library(radiant.data)
dat <- read.csv("yeast_growth.tsv", header=TRUE, row.names=1,sep = "\t")
tpm <- dat[, 3:5539]
condition = dat$Nutrient == 'Phosphate'
test = t.test(tpm[condition],tpm[!condition])$p.value < 0.05
tpm2 <- log(tpm+1)
alltests = pbapply :: pbapply(tpm2,2,function(x) t.test(x[condition],x[!condition])$p.value)
sig <- test.dat$tx[test.dat$pval < 0.05/5537]
alltests %>%
  as.data.frame() %>%
  set_colnames("pval") %>%
  rownames_to_column("tx") %>%
  arrange(pval) %>%
  head()

In [ ]:
#visualize DE genes
most_sig <- dat['YAR071W_PHO11']
most_sig$log_x <- log(most_sig$YAR071W_PHO11+1)
most_sig$condition <- dat$Nutrient
library(ggplot2)
ggplot(most_sig, aes(x = condition, y = log_x)) +
  geom_point() +
  xlab("Nutrient conditions") +
  ylab("Log Transformed TPM Values") +
  ggtitle("Expression across conditions") +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))
    
    
ggplot(most_sig, aes(x = rownames(most_sig), y = log_x, fill = condition)) +
  geom_bar(stat = "identity") +
  labs(x = "Level", y = "Expression", title = "Gene Expression Across Levels") +
  xlab("Sample ID") +
  ylab("Log Transformed TPM Values") +
  ggtitle("Expression across samples") +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))

In [ ]:
#co-expression clustering
variances = np.var(growth.iloc[:, 3:], axis=0)
gene_name = growth.columns[3:]
gene_var = pd.DataFrame({'var': variances, 'name': gene_name})
upper_threshold = gene_var['var'].quantile(0.75)
top_quantile = gene_var[gene_var['var'] >= upper_threshold]
top_name = top_quantile.index

In [ ]:
#clustering and visualization
library(ClassDiscovery)
library(dbscan)
tx.matrix <- as.matrix(tpm2[,topVar])
distance <- distanceMatrix(tx.matrix,"pearson")
opt <- optics(distance, minPts = 5)
test = extractXi(opt,xi=0.05)
clusters <- data.frame(sort(table(test$cluster),decreasing = T))
clusters_new <- clusters %>%
  slice(2:(n() - 1))

ggplot(clusters_new, aes(x=Var1, y=Freq)) + 
  geom_bar(stat="identity") +
  labs(x="Cluster", y="Number of Genes") +
  theme_classic()

In [ ]:
#get largest cluster and visualize the expression across samples
cluster_indices <- which(test$cluster == 9)
cluster9 <- topVar[cluster_indices]
write.table(cluster9, "cluster9.txt", sep = "\t", row.names = FALSE, col.names = FALSE)
with open("cluster9.txt", "r") as file:
    cluster9 = [line.strip() for line in file]
cluster_9_dat = growth[cluster9]
cluster9_sorted = cluster_9_dat.sort_values('SampleID')
cluster9_sorted.set_index('SampleID', inplace=True)
for column in cluster9_sorted.columns:
    plt.plot(cluster9_sorted.index, cluster9_sorted[column], label=column)
    
#plt.legend()
plt.xlabel('SampleID')
plt.xticks(rotation=90)
plt.ylabel('Expression')
plt.show()

In [ ]:
#gene set enrichment
shiny = pd.read_csv("/Users/yeye/Downloads/enrichment_all.csv", sep=",")